# Load and Preprocess

In [ ]:
# Import necessary libraries
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, measure, morphology
from skimage.morphology import skeletonize, label
from skimage.measure import regionprops
from skimage.feature import local_binary_pattern

# Load and preprocess the image
def load_and_preprocess_image(image_path):
    # Load the fundus image
    image = cv2.imread(image_path)
    
    # Convert to grayscale to detect the fundus boundary
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Threshold the grayscale image to create a binary mask
    _, mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    
    # Find the contours of the fundus region
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw the largest contour (assumed to be the fundus) on the mask
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    
    # Apply the mask to remove the background in the original image
    image_with_black_bg = cv2.bitwise_and(image, image, mask=mask)

    # Apply denoising to reduce noise in the image
    denoised_image = cv2.fastNlMeansDenoisingColored(image_with_black_bg, None, h=10, templateWindowSize=7, searchWindowSize=21)
    
    # Extract the green channel from the denoised image
    green_channel = denoised_image[:, :, 1]
    
    # Apply a Gaussian filter to smooth the green channel
    smoothed = cv2.GaussianBlur(green_channel, (5, 5), 0)
    
    # Apply histogram equalization to enhance contrast in the green channel
    equalized = cv2.equalizeHist(smoothed)
    
    return image_with_black_bg, green_channel, equalized

# Load image
image_path = r"..\..\Data\APTOS-2019 Dataset\train_images\train_images\1b8ad0afe9fb.png"

original_image, green_channel, equalized_image = load_and_preprocess_image(image_path)

# Visualize the original, green channel, and equalized image
plt.figure(figsize=(15,5))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(green_channel, cmap='gray')
plt.title("Green Channel")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(equalized_image, cmap='gray')
plt.title("Equalized Image")
plt.axis('off')

plt.show()

#  Blood Vessel Detection and Counting

In [ ]:
# Blood vessel detection using Canny edge detection and morphological operations
def detect_blood_vessels(equalized_image):
    # Use Canny Edge Detection for blood vessels
    edges = cv2.Canny(equalized_image, threshold1=30, threshold2=80)
    
    # Morphological operations to enhance vessels
    kernel = np.ones((3, 3), np.uint8)
    vessels = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    return vessels

# Detect blood vessels
vessels = detect_blood_vessels(equalized_image)

# Detect and count blood vessel segments
def count_blood_vessels(vessels):
    # Find connected components of the blood vessels
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(vessels, connectivity=8)
    
    # Filter out small components (noise)
    filtered_labels = [centroid for i, centroid in enumerate(centroids) if stats[i][4] > 50]  # area threshold of 50
    
    return len(filtered_labels), filtered_labels

# Count blood vessels
vessel_count, vessel_centroids = count_blood_vessels(vessels)

# Visualize vessel centroids on the original image
marked_image = original_image.copy()
for centroid in vessel_centroids:
    x, y = int(centroid[0]), int(centroid[1])
    cv2.circle(marked_image, (x, y), 10, (255, 0, 0), 2)  # Blue circle for vessels

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot the marked image with the vessel count
axs[0].imshow(cv2.cvtColor(marked_image, cv2.COLOR_BGR2RGB))
axs[0].set_title(f"Blood Vessel Count: {vessel_count}")
axs[0].axis('off')

# Plot the detected blood vessels
axs[1].imshow(vessels, cmap='gray')
axs[1].set_title("Detected Blood Vessels")
axs[1].axis('off')

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()


# Exudate Detection

In [ ]:
# Detect bright spots (potential exudates) using thresholding and labeling
def detect_exudates(gray_image):
    # Apply Otsu's thresholding to detect bright regions
    threshold = filters.threshold_otsu(gray_image)
    binary_exudates = gray_image > threshold
    
    # Label connected regions (exudates)
    labels = measure.label(binary_exudates, connectivity=2)
    
    # Get region properties (size, shape, etc.)
    regions = measure.regionprops(labels)
    
    return binary_exudates, labels, regions

# Function to mark detected exudates on the original image
def mark_exudates_on_image(image, exudate_regions):
    marked_image = image.copy()

    # Draw circles around detected exudates
    for region in exudate_regions:
        # Get the centroid of each exudate region
        y, x = region.centroid
        
        # Draw a circle around the centroid on the original image
        cv2.circle(marked_image, (int(x), int(y)), 15, (255, 0, 0), 2)  # Blue circles

    return marked_image

# Load and preprocess the image (assuming it's already preprocessed)
image_with_black_bg, green_channel, equalized_image = load_and_preprocess_image(image_path)

# Detect exudates in the equalized green channel image
binary_exudates, labels, exudate_regions = detect_exudates(equalized_image)

# Mark exudates on the original image
marked_image = mark_exudates_on_image(image_with_black_bg, exudate_regions)

# Display the marked image with exudates
plt.figure(figsize=(6, 6))
plt.imshow(cv2.cvtColor(marked_image, cv2.COLOR_BGR2RGB))
plt.title(f"Exudates Detected: {len(exudate_regions)}")
plt.axis('off')
plt.show()

# Print the count of detected exudates
print(f"Number of detected exudates: {len(exudate_regions)}")

# Bifurcation Detection

In [ ]:
# Function to skeletonize the blood vessel map and find bifurcations
def detect_bifurcations(vessel_image):
    # Skeletonize the vessel image (reduce vessels to 1-pixel width)
    skeleton = skeletonize(vessel_image // 255)
    
    # Find branch points by counting pixels with more than two neighbors
    neighbors = np.zeros(skeleton.shape, dtype=int)
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx != 0 or dy != 0:
                neighbors += np.roll(np.roll(skeleton, dx, axis=0), dy, axis=1)
    
    # Identify bifurcation points (where number of neighbors is greater than 2)
    bifurcations = (neighbors > 2).astype(np.uint8)
    
    # Label bifurcations and count them
    bifurcation_labels = label(bifurcations)
    bifurcation_count = len(np.unique(bifurcation_labels)) - 1  # Subtract 1 for background
    
    return bifurcations, bifurcation_count

# Function to mark detected bifurcations on the original image
def mark_bifurcations_on_image(image, bifurcation_points):
    marked_image = image.copy()
    
    # Find coordinates of bifurcation points
    bifurcation_coords = np.column_stack(np.nonzero(bifurcation_points))
    
    # Draw circles on the original image at bifurcation points
    for y, x in bifurcation_coords:
        cv2.circle(marked_image, (x, y), 5, (0, 255, 0), 2)  # Green circles for bifurcations
    
    return marked_image

# Load and preprocess the image (assuming it's already preprocessed)
image_with_black_bg, green_channel, equalized_image = load_and_preprocess_image(image_path)

# Detect vessels (assuming `vessels` is the binary blood vessel map from earlier)
vessels = (equalized_image > filters.threshold_otsu(equalized_image)).astype(np.uint8) * 255

# Detect bifurcations in the skeletonized vessel image
bifurcations, bifurcation_count = detect_bifurcations(vessels)

# Mark bifurcations on the original image
marked_image = mark_bifurcations_on_image(image_with_black_bg, bifurcations)

# Display the marked image with bifurcations
plt.figure(figsize=(6, 6))
plt.imshow(cv2.cvtColor(marked_image, cv2.COLOR_BGR2RGB))
plt.title(f"Bifurcations Detected: {bifurcation_count}")
plt.axis('off')
plt.show()

# Print the count of detected bifurcations
print(f"Number of detected bifurcations: {bifurcation_count}")

# Large Oedema Detection

In [ ]:
# Function to detect large oedema spots
def detect_large_oedema_spots(gray_image, min_area=5000, threshold_factor=1.5):
    # Step 1: Apply Gaussian blur to smooth the image and reduce noise
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    
    # Step 2: Apply a threshold to detect bright regions (objects)
    threshold_value = threshold_factor * cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    binary_objects = blurred_image > threshold_value
    
    # Step 3: Find contours (outlines) of the objects
    contours, _ = cv2.findContours(binary_objects.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Step 4: Create an empty image for large objects
    large_objects = np.zeros_like(binary_objects, dtype=np.uint8)
    large_object_count = 0
    
    # Step 5: Filter and count objects based on area
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= min_area:
            cv2.drawContours(large_objects, [contour], -1, 1, thickness=cv2.FILLED)  # Fill in the large object
            large_object_count += 1
    
    return large_objects, large_object_count, contours

# Function to mark detected large oedema spots on the original image
def mark_large_oedema_on_image(image, contours):
    marked_image = image.copy()
    
    # Draw circles around the large oedema spots
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= 1000:  # Filtering based on the minimum area
            # Get the center and radius of the minimum enclosing circle
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            # Draw the circle on the original image
            cv2.circle(marked_image, center, radius, (0, 0, 255), 2)  # Red circles for oedema spots
    
    return marked_image

# Load and preprocess the image (assuming it's already preprocessed)
image_with_black_bg, green_channel, equalized_image = load_and_preprocess_image(image_path)

# Detect large oedema spots in the green channel
large_oedema_spots, oedema_count, contours = detect_large_oedema_spots(green_channel)

# Mark oedema spots on the original image
marked_image = mark_large_oedema_on_image(image_with_black_bg, contours)

# Display the marked image with large oedema spots
plt.figure(figsize=(6, 6))
plt.imshow(cv2.cvtColor(marked_image, cv2.COLOR_BGR2RGB))
plt.title(f"Large Oedema Spots Detected: {oedema_count}")
plt.axis('off')
plt.show()
